In [52]:
import sklearn
import numpy as np
import pandas as pd
#from tensorflow.keras.preprocessing.text import Tokenizer

In [14]:
data_train = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')
X = data_train.text
y = data_train.rating
X_final_test = data_test.text

In [15]:
# Funcion para limpieza de texto
import re

def limpiar(list):
    my_valoraciones_new = []
    for x in list:
        if type(x) == str:
            x = x.lower()
            x = re.sub(r"[n]*$", " ", x)
            x = re.sub(r"that's","that is",x)
            x = re.sub(r"there's","there is",x)
            x = re.sub(r"what's","what is",x)
            x = re.sub(r"where's","where is",x)
            x = re.sub(r"it's","it is",x)
            x = re.sub(r"who's","who is",x)
            x = re.sub(r"i'm","i am",x)
            x = re.sub(r"she's","she is",x)
            x = re.sub(r"he's","he is",x)
            x = re.sub(r"they're","they are",x)
            x = re.sub(r"who're","who are",x)
            x = re.sub(r"ain't","am not",x)
            x = re.sub(r"wouldn't","would not",x)
            x = re.sub(r"shouldn't","should not",x)
            x = re.sub(r"can't","can not",x)
            x = re.sub(r"couldn't","could not",x)
            x = re.sub(r"won't","will not",x)
            x = re.sub(r"\W"," ",x)
            x = re.sub(r"\d"," ",x)
            x = re.sub(r"\s+[a-z]\s+"," ",x)
            x = re.sub(r"\s+[a-z]$"," ",x)
            x = re.sub(r"^[a-z]\s+"," ",x)
            x = re.sub(r"\s+"," ",x)
            my_valoraciones_new.append(x)
    return my_valoraciones_new

In [16]:
# Limpio la X train y la X test con re
X_limpio = limpiar(X)
X_final_test_limpio = limpiar(X_final_test)

In [ ]:
# Vectorizar los datos de texto utilizando CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(X_limpio)

In [51]:
#Vectorizo el X train
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
'''
TF (Term Frequency - Frecuencia de Término): Mide la importancia de una palabra en un documento específico. 
Cuantas más veces aparece una palabra en un documento, mayor es su peso TF.
vectorizer = TfidfVectorizer(max_features = 50)
IDF (Inverse Document Frequency - Frecuencia Inversa de Documento): Mide la importancia de una palabra en el contexto 
de un conjunto de documentos. Cuanto menos común es una palabra en todos los documentos, mayor es su peso IDF
TF-IDF Score: El puntaje TF-IDF de una palabra en un documento se obtiene multiplicando su valor TF por su valor IDF. 
Esto da como resultado un puntaje que refleja la importancia de la palabra tanto en el documento 
como en el conjunto de documentos. La fórmula general para el puntaje TF-IDF de una palabra en un documento es:
TF-IDF(t,d,D)=TF(t,d)×IDF(t,D)TF-IDF(t,d,D)=TF(t,d)×IDF(t,D)
Vectorización: Después de calcular los puntajes TF-IDF para todas las palabras en un conjunto de documentos, 
cada documento se representa como un vector numérico donde cada dimensión del vector corresponde a una palabra 
y el valor en esa dimensión es el puntaje TF-IDF de esa palabra en el documento. 
Esto crea una representación vectorial densa de los documentos que se puede utilizar en algoritmos de aprendizaje automático.

En resumen, el TF-IDF Vectorizer transforma documentos de texto en vectores numéricos ponderados que capturan 
la importancia relativa de las palabras en cada documento en el contexto de un conjunto de documentos. 
Esto es útil para tareas como [[[recuperación de información, clasificación de texto y agrupación de documentos]]], 
ya que permite a los algoritmos de aprendizaje automático trabajar con datos de texto de manera efectiva.

PARAMETROS: 
max_features: máximo numero de palabras a obtener
min_df/max_df : frecuencia de aparición máxima o mínima en los documentos
stop_words: palabras a eliminar
binary (default=False): Si True,resultado 1 o 0, indicando la presencia o ausencia de una palabra en un documento.

'''
vectorizer = TfidfVectorizer(max_features= 1000) # max_features, min/max_df, stop_words, binary(False)

X = vectorizer.fit(X_limpio,X_final_test_limpio)
#Vectorizo la X
X_final_test = vectorizer.transform(X_limpio).toarray()
X_final_test = vectorizer.transform(X_final_test_limpio).toarray()

In [18]:
# Divido el dataset en entrenamiento y validación
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [23]:
# Entrenamiento con random forest:
from sklearn.ensemble import RandomForestClassifier
'''
Crea arboles con columnas y filas aleatorias del dataset y va dividiendo en ramas hasta que llega al final
n_estimators[d=100]: Numero de arboles(aleatorios) en el bosque
max_depth[d=None, hasta min_samples_split]: Profundidad máxima de los árboles.
min_samples_split[d=2]: Número mínimo de muestras requeridas para dividir un nodo interno.
min_samples_leaf[d=1]: Número mínimo de muestras requeridas para ser una hoja
max_features[d=raiz cuadrada total caracteristicas]: Número de características a considerar para encontrar la mejor división en cada árbol

'''
random_forest_classifier = RandomForestClassifier(n_estimators=200, random_state=42)
random_forest_classifier.fit(X_train, y_train)
y_pred_forest = random_forest_classifier.predict(X_test)
print(sklearn.metrics.f1_score(y_test, y_pred_forest, average='micro'))
y_final_randomforest = random_forest_classifier.predict(X_final_test)
#Con 1000 palabras 0,68, 2000: 0.67, 500: 0,678 , 50 = 0.670
#1000 palabras: 50 estimators:0,674,

0.6716723549488055


### Naive Bayes
Clasificación y análisis de texto, especialmente NPL. 
Funciona en base al teorema de Bayes, que es una regla de probabilidad condicional, 
y asume independencia condicional entre las características, lo que le da el nombre de "Naive"
calcular la probabilidad de una hipótesis (clase) basada en la evidencia observada (características)

In [26]:
# SOLO PARA DATOS CONTINUOS
# el clasificador Naive Bayes Gaussiano asume que las características son continuas y siguen una distribución gaussiana
from sklearn.naive_bayes import GaussianNB

naive_gaussian = GaussianNB()
naive_gaussian.fit(X_train, y_train)
y_pred_naive_g = naive_gaussian.predict(X_test)
print(sklearn.metrics.f1_score(y_test, y_pred_naive_g, average='micro'))
y_final_naive_g = naive_gaussian.predict(X_final_test)

0.2887372013651877


In [35]:
# Entrenamiento con MultinomialNB
from sklearn.naive_bayes import MultinomialNB
#alpha[d=1] = suaviza los datos para evitar la probabilidad nula 
naive_bayes_classifier = MultinomialNB(alpha=0)
naive_bayes_classifier.fit(X_train, y_train)
y_pred_naive_m = naive_bayes_classifier.predict(X_test)
print(sklearn.metrics.f1_score(y_test, y_pred_naive_m, average='micro'))
y_final_naive_m = naive_bayes_classifier.predict(X_final_test)
#Con 1000 palabras 0,679, 2000: 0,670, 500:0.677, 50; 0,66
# Con alpha 0 : 0.693


0.6935153583617747


C:\Users\amand\anaconda3\Lib\site-packages\sklearn\naive_bayes.py:624: FutureWarning: The default value for `force_alpha` will change to `True` in 1.4. To suppress this warning, manually set the value of `force_alpha`.
  warnings.warn(
C:\Users\amand\anaconda3\Lib\site-packages\sklearn\naive_bayes.py:633: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10. Use `force_alpha=True` to keep alpha unchanged.
  warnings.warn(


In [40]:
#Entrenamiento con BernoulliNB
# [[[BINARIO]]]
from sklearn.naive_bayes import BernoulliNB

bernoulli = BernoulliNB()
bernoulli.fit(X_train, y_train)
y_pred_naive_b = bernoulli.predict(X_test)
print(sklearn.metrics.f1_score(y_test, y_pred_naive_b, average='micro'))
y_final_naive_b = bernoulli.predict(X_final_test)


0.6450511945392492


In [43]:
# Entrenamiento con Logistic Regression
from sklearn.linear_model import LogisticRegression
# En Regresión Logística, se utiliza la función logística (también llamada función sigmoide) 
# para modelar la probabilidad de que una instancia pertenezca a una clase particular.
# La función sigmoide transforma una combinación lineal de las características en un valor en el rango de 0 a 1.
# Para usarlo para mas de 2 clases se usa multi_class='multinomial', solver='lbfgs'
LR = LogisticRegression(multi_class='multinomial')
LR.fit(X_train, y_train)
y_pred_regres = LR.predict(X_test)
print(sklearn.metrics.f1_score(y_test, y_pred_regres, average='micro'))
y_final_regres = LR.predict(X_final_test)
# 1000:0.698, 2000: 0,698, 500: 0.6969, 50: 0,679

0.7044368600682593


In [47]:
# Entrenamiento con Knn
from sklearn.neighbors import KNeighborsClassifier
# Para clasificacion y regresion
# weights='unifor,', 'distance' Peso de los vecinos(uniforme o por distancia)

knn = KNeighborsClassifier(n_neighbors=3).fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)
print(sklearn.metrics.f1_score(y_test, y_pred_knn, average='micro'))
y_final_knn = knn.predict(X_final_test)
# 1000:0.414, 2000: 0.652, 500: 0.640, 50: 0,645
#nn=3: 0'65,

0.6505119453924915


In [49]:
# Entrenamiento con arboles de decision
from sklearn.tree import DecisionTreeClassifier
'''
    criterion: El criterio utilizado para medir la calidad de una división. 
    Puede ser 'gini' para el índice Gini o 'entropy' para la ganancia de información. 
    El índice Gini es el valor por defecto.

    max_depth: La profundidad máxima del árbol. Puedes establecer un valor entero para limitar 
    la profundidad del árbol. Si se establece en None, el árbol se expandirá hasta que todas las hojas sean puras 
    o hasta que contengan menos de min_samples_split muestras.

    min_samples_split: El número mínimo de muestras requeridas para dividir un nodo interno. 
    Puede ser un valor entero o un número decimal.

    min_samples_leaf: El número mínimo de muestras requeridas en una hoja. 
    Puede ser un valor entero o un número decimal.

    max_features: El número máximo de características que se consideran al hacer una división. 
    Puede ser un valor entero o un número decimal.
'''
clf = DecisionTreeClassifier(criterion='entropy', max_depth=None, min_samples_split=2, min_samples_leaf=1)
clf = clf.fit(X_train,y_train)
y_pred_tree = clf.predict(X_test)
print(sklearn.metrics.f1_score(y_test, y_pred_tree, average='micro'))
y_final_tree = clf.predict(X_final_test)
# 1000: 0.666, 2000: 0.658, 500: 0.662, xdefect= 0.662

0.6027303754266211


In [42]:
# RESULTADO
resul = pd.DataFrame()
resul['id']= data_test.id
resul['rating'] = y_final_regres
resul.to_csv('resultado_c.csv', index=False)
resul

,id,rating
0,5384,5
1,3529,5
2,3640,5
3,1263,5
4,7483,5
...,...,...
1949,3734,5
1950,4518,5
1951,1358,5
1952,4348,5
